In [ ]:
import deepxde as dde
import numpy as np
import scipy.io
import os
import urllib.request
import tensorflow as tf

In [ ]:
# --- 1. 定义被“篡改”过的物理方程 (PDE) ---
def pde(x, y):
    """
    定义纳维-斯托克斯方程组。
    x: 输入张量, x[:, 0] 是 x 坐标, x[:, 1] 是 y 坐标, x[:, 2] 是 t 时间。
    y: 输出张量, y[:, 0] 是 u, y[:, 1] 是 v, y[:, 2] 是 p。
    """
    u, v, p = y[:, 0:1], y[:, 1:2], y[:, 2:3]
    
    # 使用 dde.grad.jacobian 和 dde.grad.hessian 计算各阶导数
    du_t = dde.grad.jacobian(y, x, i=0, j=2)
    du_x = dde.grad.jacobian(y, x, i=0, j=0)
    du_y = dde.grad.jacobian(y, x, i=0, j=1)
    du_xx = dde.grad.hessian(y, x, component=0, i=0, j=0)
    du_yy = dde.grad.hessian(y, x, component=0, i=1, j=1)

    dv_t = dde.grad.jacobian(y, x, i=1, j=2)
    dv_x = dde.grad.jacobian(y, x, i=1, j=0)
    dv_y = dde.grad.jacobian(y, x, i=1, j=1)
    dv_xx = dde.grad.hessian(y, x, component=1, i=0, j=0)
    dv_yy = dde.grad.hessian(y, x, component=1, i=1, j=1)

    dp_x = dde.grad.jacobian(y, x, i=2, j=0)
    dp_y = dde.grad.jacobian(y, x, i=2, j=1)

    # C1 和 C2 是我们要反演的未知参数
    # 我们用 dde.Variable 来定义它们，并给一个初始猜测值
    C1 = dde.Variable(1.0)
    C2 = dde.Variable(0.01)

    # 1. 连续性方程
    f_continuity = du_x + dv_y
    # 2. x方向动量方程
    f_momentum_x = du_t + C1 * (u * du_x + v * du_y) + dp_x - C2 * (du_xx + du_yy)
    # 3. y方向动量方程
    f_momentum_y = dv_t + C1 * (u * dv_x + v * dv_y) + dp_y - C2 * (dv_xx + dv_yy)
    
    return [f_continuity, f_momentum_x, f_momentum_y]

In [ ]:
# --- 数据准备 ---
# 检查数据文件是否存在，如果不存在则自动下载
data_file = "cylinder_nektar_wake.mat" # 修正：使用正确的数据文件
if not os.path.exists(data_file):
    print(f"Downloading {data_file}...")
    # 更新为更稳定的原始文件链接
    url = f"https://raw.githubusercontent.com/maziarraissi/PINNs/master/main/Data/{data_file}"
    urllib.request.urlretrieve(url, data_file)
    print("Download complete.")
# 加载 .mat 数据文件
data = scipy.io.loadmat(data_file)
t = data["t"].flatten()[:, None]      # 时间点, Shape: (T, 1)
X_star = data["X_star"]               # 空间坐标点, Shape: (N, 2)
U_star = data["U_star"]               # 速度场 (u,v), Shape: (N, 2, T) in this file
p_star = data["p_star"]               # 压力场, Shape: (N, T)

N = X_star.shape[0]
T = t.shape[0]

# 将数据整理成 (x, y, t) 和 (u, v, p) 的配对形式
xx = np.tile(X_star[:, 0:1], (1, T))
yy = np.tile(X_star[:, 1:2], (1, T))
tt = np.tile(t, (1, N)).T

x_train = xx.flatten()[:, None]
y_train = yy.flatten()[:, None]
t_train = tt.flatten()[:, None]

observe_xt = np.hstack((x_train, y_train, t_train))

# 修正：根据 (N, 2, T) 的结构正确提取 u 和 v
u_train = U_star[:, 0, :].T.flatten()[:, None]
v_train = U_star[:, 1, :].T.flatten()[:, None]
p_train = p_star.T.flatten()[:, None]

observe_y = np.hstack((u_train, v_train, p_train))


In [21]:
# --- 定义数据约束 ---
# 使用 PointSetBC 来告诉 DeepXDE，这些数据点是我们的约束
# 这是一个逆问题，所以我们只有数据点，没有传统的边界条件
observe_bc = dde.icbc.PointSetBC(observe_xt, observe_y)

# 组合成 DeepXDE 的数据对象
# 这里的 10000 是在整个时空域内额外采样的物理点数量
# 我们只需要一个时空域来告诉DeepXDE问题的范围
# 空间是矩形 [-5, 15] x [-5, 5], 时间从 [0, 20]
space_domain = dde.geometry.Rectangle([-5, -5], [15, 5])
time_domain = dde.geometry.TimeDomain(0, 20)
geom = dde.geometry.GeometryXTime(space_domain, time_domain)

data = dde.data.TimePDE(
    geom,
    pde,
    [observe_bc],
    num_domain=10000,
    train_distribution="pseudo",
    solution=None,
)

# --- 网络、模型和训练 ---
net = dde.nn.FNN([3] + [100] * 8 + [3], "tanh", "Glorot normal")
model = dde.Model(data, net)

# 告诉模型，我们定义的 C1 和 C2 是需要被训练的外部变量
external_trainable_variables = [dde.Variable(1.0), dde.Variable(0.01)]
variable = dde.callbacks.VariableValue(
    external_trainable_variables, period=1000, filename="variables.dat"
)

# 编译并训练模型
model.compile(
    "adam",
    lr=1e-3,
    external_trainable_variables=external_trainable_variables,
)

print("Starting training...")
losshistory, train_state = model.train(iterations=100000, callbacks=[variable])

print("\nTraining finished!")
dde.saveplot(losshistory, train_state, issave=True, isplot=True)

# 从文件中读取最终的参数值
final_params = np.loadtxt("variables.dat")
final_c1 = final_params[-1, 1]
final_c2 = final_params[-1, 2]

print(f"\n最终猜测 c1: {final_c1:.4f} (真实值: 1.0)")
print(f"最终猜测 c2: {final_c2:.4f} (真实值: 0.01)")


Compiling model...
Building feed-forward neural network...
'build' took 0.119559 s

'compile' took 2.153975 s

Starting training...
Training model...

Step      Train loss                                  Test loss                                   Test metric
0         [1.78e-03, 2.94e-03, 2.78e-03, 3.20e-01]    [1.78e-03, 2.94e-03, 2.78e-03, 3.20e-01]    []  


: 